<a href="https://colab.research.google.com/github/romauligraciella/topic-level-sentiment-analysis/blob/main/Topic_Level_Sentiment_Preprocessing_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


# Scraping

In [ ]:
!pip install google_play_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.7 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import app, Sort, reviews
import pandas as pd
import numpy as np

In [ ]:
#Scrape desired number of reviews

app_reviews,continuation_token = reviews(
    'app.bpjs.mobile',
    lang='id', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT you can use Sort.NEWEST to get newst reviews
    count=100000, # defaults to 100
    filter_score_with=None # defaults to None(means all score) Use 1 or 2 or 3 or 4 or 5 to select certain score
)

In [ ]:
df_review = pd.DataFrame(np.array(app_reviews),columns=['review'])
df_review = df_review.join(pd.DataFrame(df_review.pop('review').tolist()))

df_review.to_csv('/content/drive/MyDrive/FIX_SKRIPSI/Data/most_relevant_reviews_MobileJKN.csv', index=False)

# Preprocessing

In [ ]:
import nltk
nltk.download('all')
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/FIX_SKRIPSI/Data/most_relevant_reviews_MobileJKN.csv', usecols=["userName", "score",	"content"])

In [ ]:
import re
import pandas as pd

# Load kamus alay
kamus_alay = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
nor_dict = {row[0]: row[1] for _, row in kamus_alay.iterrows()}

def clean_text_1(text):
    # 01 Konversi ke huruf kecil
    text = text.strip().lower()

    # 02 Menghilangkan tautan
    text = re.sub(r'(www\.[^\s]+)|(https?://[^\s]+)', '', text)

    # 03 Hapus @mention
    text = re.sub(r'@\w+', '', text)

    # 04 Hilangkan '#' tapi simpan katanya
    text = re.sub(r'#(\w+)', r'\1', text)

    # 05 Hapus emoji dan unicode simbol
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols
        u"\U0001F680-\U0001F6FF"  # transport
        u"\U0001F1E0-\U0001F1FF"  # flags
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # 06 Normalisasi tanda baca ganda (misal "???" jadi "?")
    text = re.sub(r'([!?.,])\1+', r'\1', text)

    # 07 Buang karakter selain huruf, spasi, dan tanda baca yang disimpan
    text = re.sub(r"[^a-zA-Z\s!?.,]", ' ', text)

    # 08 Buang spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenisasi
    tokens = []
    for word in text.split():
        # Hapus huruf berulang lebih dari dua kali
        word = re.sub(r"(.)\1{2,}", r"\1\1", word)

        # Ganti jika ada di kamus alay
        if word in nor_dict:
            word = nor_dict[word]

        # Validasi minimal 3 huruf
        if word == "rt" or not re.match(r'^[a-zA-Z!?.,]{3,}$', word):
            continue
        tokens.append(word)

    return ' '.join(tokens)

<ipython-input-23-effd0790d7c0>:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nor_dict = {row[0]: row[1] for _, row in kamus_alay.iterrows()}


In [ ]:
## Untuk cTFIDF
def clean_text_2(text):
    import re
    # 01 Konversi ke huruf kecil
    text = text.lower()
    # 02 Menghilangkan tanda baca (kecuali spasi dan huruf/angka)
    text = re.sub(r'[^\w\s]', ' ', text)
    # 03 Menghilangkan angka
    text = re.sub(r'\d+', '', text)
    # 04 Menghilangkan spasi ganda
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

additional_stopwords = ['nya', 'ya', 'kalo', 'tolong', 'gue', 'gua', 'kali', 'sih',
                        'kasih', 'terima', 'terimakasih', 'makasih', 'min', 'pas',
                        'banget', 'saya', 'saat', 'deh', 'dll', 'nih', 'donk',
                        'ny', 'si', 'eh', 'tu', 'tuh', 'mah', 'loh', 'aja', 'mulu',
                        'bangsat', 'tai','goblok', 'tolol', 'ga becus', 'blok',
                        'security', 'jokowi', 'ramah','mau tanya']
stop_words = stopwords.words('indonesian') + additional_stopwords + stopwords.words('english')
stop_words = set(stop_words)

def remove_stop_words(s):
    return ' '.join(word for word in s.split() if word not in stop_words)

In [ ]:
dataset["text_1"] = dataset["content"].astype(str)
dataset["text_1"] = dataset["text_1"].map(lambda x: clean_text_1(x))
dataset["text_2"] = dataset["text_1"].map(lambda x: clean_text_2(x))
dataset["text_2"] = dataset["text_2"].apply(lambda x: remove_stop_words(x))
dataset = dataset[dataset["text_2"].apply(lambda x: len(x.split()) >=3)]

In [ ]:
len(dataset)

88844

In [ ]:
dataset

,userName,content,score,text_1,text_2
0,Pengguna Google,"Aplikasi gak jelas, sering bug, sering error, ...",1,"aplikasi enggak jelas, sering bug, sering erro...",aplikasi bug error ribet buka aplikasi log dip...
1,Pengguna Google,saya mau daftar antri lagi buat periksa tp sel...,1,saya mau daftar antri lagi buat periksa tapi s...,daftar antri periksa gabisa muncul antrean pol...
2,Pengguna Google,"Aplikasi Rusak!! Issue critical gak fix""dari t...",1,aplikasi rusak! issue critical enggak fix dari...,aplikasi rusak issue critical fix daftar kirim...
3,Pengguna Google,"Kirain cuman sendiri yg OTPnya ga ke kirim, li...",1,mengira cuman sendiri yang otpnya enggak kirim...,cuman otpnya kirim lihat ulasan ngalamin pakai...
4,Pengguna Google,"Aplikasi jelek, sebenernya dibuat untuk memuda...",1,"aplikasi jelek, sebenarnya dibuat untuk memuda...",aplikasi jelek memudahkan menyulitkan masyarak...
...,...,...,...,...,...
99994,Pengguna Google,Bintang 2 aja.. G puas karna sering update..,2,bintang aja. enggak puas karena sering update.,bintang puas update
99995,harir turmudzi,Sebentar² harus update jadi ribet,1,sebentar harus update jadi ribet,sebentar update ribet
99996,Pari Yanto,Penuh kepahitan dan penderitaan hidup.Masa kec...,5,penuh kepahitan dan penderitaan hidup.masa kec...,penuh kepahitan penderitaan hidup anak meranta...
99997,Afred Pake,Mantap aplikasinya.. Sudah tidak ribet urusannya,5,mantap aplikasinya. sudah tidak ribet urusannya,mantap aplikasinya ribet urusannya


In [ ]:
dataset.to_csv('/content/drive/MyDrive/FIX_SKRIPSI/Data/cleaned_data_MobileJKN.csv', index=False)

In [ ]:
dataset['label'] = None

In [ ]:
negatif_mask = dataset['text_1'].str.contains('sangat jelek|sangat buruk|kecewa|memalukan|buruk|sampah|tidak membantu|mempersulit', case=False, na=False)
negatif_indices = dataset[negatif_mask].index[:4442]
dataset.loc[negatif_indices, 'label'] = 0

positif_mask = dataset['text_1'].str.contains('lancar|lebih cepat|sangat cepat|kemudahan|sangat membantu|sangat bagus|tidak mengecewakan', case=False, na=False)
score_mask = dataset['score'].isin([4, 5])
positif_indices = dataset[positif_mask & score_mask].index[:4442]  # ambil 4442 data pertama
dataset.loc[positif_indices, 'label'] = 1

In [ ]:
dataset['label'].unique()

array([None, 0, 1], dtype=object)

In [ ]:
dataset['label'].value_counts(dropna=False)

,count
label,
None,79971
1,4442
0,4431


In [ ]:
dataset.to_csv('/content/drive/MyDrive/FIX_SKRIPSI/Data/dataset_MobileJKN.csv', index=False)